In [1]:
import os

os.chdir("..")
print(os.path.abspath(os.curdir))

d:\Data Kuliah\Semester 4\Seminar 2\Definition Extraction\drone-definition


In [3]:
!pip install pandas

  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   --------------------- ------------------ 6.3/11.5 MB 39.2 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 40.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------- ----------------- 7.1/12.6 MB 58.8 MB/s eta 0:00:01
   -------------------------------- ------- 10.2/12.6 MB 26.3 MB/s eta 0:00:01
   ---------------------------------------  12.6/12.6 MB 21.2 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 18.4 MB/s eta 0:00:00
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)


In [8]:
import pandas as pd
train_path = os.path.join('dataset', 'airdata_train_70.csv')
test_path = os.path.join('dataset', 'airdata_test_70.csv')

train_df = pd.read_csv(train_path, encoding="utf-8", delimiter=";")
test_df = pd.read_csv(test_path)

In [9]:
train_df

,sentence_id,words,labels
0,1,Aircraft,B-COMPONENT
1,1,is,O
2,1,in,O
3,1,Attitude,B-STATE
4,1,mode,I-STATE
...,...,...,...
12112,2192,with,O
12113,2192,caution,O
12114,2203,QuickShot,B-STATE
12115,2203,has,O


In [10]:
merge_df = pd.concat([train_df, test_df])
merge_df

,sentence_id,words,labels
0,1,Aircraft,B-COMPONENT
1,1,is,O
2,1,in,O
3,1,Attitude,B-STATE
4,1,mode,I-STATE
...,...,...,...
5066,2272,.,O
5067,2272,Please,O
5068,2272,fly,O
5069,2272,with,O


In [ ]:
term_list = []
for i, (sent_id, word, label) in merge_df.iterrows():
    curr_term = ""
    curr_type = ""
    curr_sid = sent_id
    if label == 'O':
        if curr_term:
            term_list.append([curr])

sentence_id              1
words             Aircraft
labels         B-COMPONENT
Name: 0, dtype: object
sentence_id     1
words          is
labels          O
Name: 1, dtype: object
sentence_id     1
words          in
labels          O
Name: 2, dtype: object
sentence_id           1
words          Attitude
labels          B-STATE
Name: 3, dtype: object
sentence_id          1
words             mode
labels         I-STATE
Name: 4, dtype: object
sentence_id     1
words          so
labels          O
Name: 5, dtype: object
sentence_id       1
words          that
labels            O
Name: 6, dtype: object
sentence_id     1
words          it
labels          O
Name: 7, dtype: object
sentence_id       1
words          will
labels            O
Name: 8, dtype: object
sentence_id      1
words          not
labels           O
Name: 9, dtype: object
sentence_id        1
words          hover
labels             O
Name: 10, dtype: object
sentence_id    1
words          .
labels         O
Name: 11, dtype: 

In [13]:
import pandas as pd

def transform_ner_dataset(df):
    """
    Transform IOB2-formatted NER dataset into entity spans and sentences dataframes.
    
    Parameters:
    df (pandas.DataFrame): Input dataframe with columns [sentence_id, word, label]
    
    Returns:
    tuple: (entity_spans_df, sentences_df)
    """
    # Initialize lists to store results
    entity_spans = []
    current_entity = []
    current_type = None
    
    # Group by sentence_id to process one sentence at a time
    sentences = {}
    
    for _, group in df.groupby('sentence_id'):
        sentence_id = group['sentence_id'].iloc[0]
        words = group['words'].tolist()
        labels = group['labels'].tolist()
        
        # Store sentence
        sentences[sentence_id] = ' '.join(words)
        
        # Process entities
        for i, (word, label) in enumerate(zip(words, labels)):
            if label == 'O':
                if current_entity:
                    entity_spans.append({
                        'sentence_id': sentence_id,
                        'entity_span': ' '.join(current_entity),
                        'entity_type': current_type
                    })
                    current_entity = []
                    current_type = None
                continue
                
            # Split IOB2 tag
            iob_prefix, entity_type = label.split('-')
            
            # Handle B- tag
            if iob_prefix == 'B':
                if current_entity:
                    entity_spans.append({
                        'sentence_id': sentence_id,
                        'entity_span': ' '.join(current_entity),
                        'entity_type': current_type
                    })
                current_entity = [word]
                current_type = entity_type
                
            # Handle I- tag
            elif iob_prefix == 'I':
                if current_type == entity_type:
                    current_entity.append(word)
                else:
                    # Handle incorrect I- tag
                    if current_entity:
                        entity_spans.append({
                            'sentence_id': sentence_id,
                            'entity_span': ' '.join(current_entity),
                            'entity_type': current_type
                        })
                    current_entity = [word]
                    current_type = entity_type
        
        # Add last entity of the sentence if exists
        if current_entity:
            entity_spans.append({
                'sentence_id': sentence_id,
                'entity_span': ' '.join(current_entity),
                'entity_type': current_type
            })
            current_entity = []
            current_type = None
    
    # Create entity spans dataframe
    entity_spans_df = pd.DataFrame(entity_spans)
    
    # Create sentences dataframe
    sentences_df = pd.DataFrame([
        {'sentence_id': sid, 'sentence': text} 
        for sid, text in sentences.items()
    ])
    
    return entity_spans_df, sentences_df


In [16]:
!pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)


In [18]:

# Example usage
# def main():
    # Read the input CSV file
# input_df = pd.read_csv('your_file.csv')

# Transform the dataset
entity_spans_df, sentences_df = transform_ner_dataset(merge_df)

# Sort dataframes by sentence_id for consistency
entity_spans_df = entity_spans_df.sort_values('sentence_id')
unique_spans_df = entity_spans_df.drop_duplicates(subset=['entity_span', 'entity_type'], keep='first')
sentences_df = sentences_df.sort_values('sentence_id')

# Save to new CSV files
entity_spans_df.to_excel('dataset/entity_spans.xlsx', index=False)
unique_spans_df.to_excel('dataset/unique_entity_spans.xlsx', index=False)
sentences_df.to_excel('dataset/sentences.xlsx', index=False)
    
# if __name__ == "__main__":
#     main()